# First version of recommendation model

### Links

Tutorial: [link](https://www.stepbystepdatascience.com/hybrid-recommender-lightfm-python)

Dataset: [instacart-market-basket-analysis](https://www.kaggle.com/datasets/psparks/instacart-market-basket-analysis)

### Prerequisites

Download dataset and place files in folder: ../data/instacart-market-basket-analysis

In [10]:
%pip install --upgrade pip setuptools wheel
%pip install numpy pandas scikit-learn optuna nbformat
%pip install --no-use-pep517 lightfm # https://github.com/lyst/lightfm/issues/687#issuecomment-1523956355
! pip install ipywidgets --upgrade


Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
Defaulting to 

In [11]:
import pickle
import sys
from os import path

import numpy as np
import pandas as pd
import optuna
import sklearn

from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split

sys.path.insert(0, '..') # важно чтобы работали импорты из корня репозитория как в инференсе
sklearn.show_versions()


System:
    python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
executable: /usr/bin/python3
   machine: Linux-5.15.146.1-microsoft-standard-WSL2-x86_64-with-glibc2.35

Python dependencies:
      sklearn: 1.2.2
          pip: 24.0
   setuptools: 69.5.1
        numpy: 1.26.0
        scipy: 1.11.3
       Cython: None
       pandas: 2.2.2
   matplotlib: 3.8.4
       joblib: 1.3.2
threadpoolctl: 3.2.0

Built with OpenMP: True

threadpoolctl info:
       user_api: blas
   internal_api: openblas
    num_threads: 20
         prefix: libopenblas
       filepath: /home/kitok/.local/lib/python3.10/site-packages/numpy.libs/libopenblas64_p-r0-0cf96a72.3.23.dev.so
        version: 0.3.23.dev
threading_layer: pthreads
   architecture: Prescott

       user_api: openmp
   internal_api: openmp
    num_threads: 20
         prefix: libgomp
       filepath: /home/kitok/.local/lib/python3.10/site-packages/scikit_learn.libs/libgomp-a34b3233.so.1.0.0
        version: None

       user_api: blas
   i

In [22]:
data_path = '../data'
dataset_path = path.join(data_path, 'instacart-market-basket-analysis')

# Маппинг user_id <-> order_id
raw_orders = pd.read_csv(path.join(dataset_path, 'orders.csv'))

# Продукты
raw_products = pd.read_csv(path.join(dataset_path, 'products.csv'))

# Маппинг order_id -> product_id
raw_order_products = pd.concat([pd.read_csv(path.join(dataset_path, 'order_products__prior.csv')),
                            pd.read_csv(path.join(dataset_path, 'order_products__train.csv'))])

# Фичи продуктов
aisles = pd.read_csv(path.join(dataset_path, 'aisles.csv'))
departments = pd.read_csv(path.join(dataset_path, 'departments.csv'))

# Чеки
raw_orders[(raw_orders["user_id"]==1)].sort_values(["order_number"])

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [23]:
# Количество юзеров
raw_orders['user_id'].unique().size

206209

In [24]:
# Айтемы
raw_products

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [25]:
# Взаимодействия
raw_order_products

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1


In [28]:
# Подрежем количество продуктов, возьмем пока топ N продуктов
top_products_count = 50
top_popular_product_counts = raw_order_products.groupby('product_id').size().sort_values()[-top_products_count:].rename('count').to_frame().reset_index()
top_popular_product_counts

,product_id,count
0,8277,87272
1,40706,88078
2,5876,91251
3,28204,92889
4,49683,99728
5,39275,105026
6,45007,109412
7,24964,113936
8,22935,117716
9,27966,142603


In [40]:
products = top_popular_product_counts.merge(raw_products, how='left', on='product_id')
products[['product_id', 'aisle_id']].merge(aisles, how='left', on='aisle_id')[['product_id', 'aisle_name']].to_csv(path.join(dataset_path, 'used_products.csv'), index=False)
products

,product_id,count,product_name,aisle_id,department_id
0,8277,87272,Apple Honeycrisp Organic,24,4
1,40706,88078,Organic Grape Tomatoes,123,4
2,5876,91251,Organic Lemon,24,4
3,28204,92889,Organic Fuji Apple,24,4
4,49683,99728,Cucumber Kirby,83,4
5,39275,105026,Organic Blueberries,123,4
6,45007,109412,Organic Zucchini,83,4
7,24964,113936,Organic Garlic,83,4
8,22935,117716,Organic Yellow Onion,83,4
9,27966,142603,Organic Raspberries,123,4


In [30]:
order_products = products[['product_id']].merge(raw_order_products, how='left', on='product_id')
order_products

,product_id,order_id,add_to_cart_order,reordered
0,8277,77,7,1
1,8277,225,6,1
2,8277,275,2,1
3,8277,323,12,1
4,8277,335,4,0
...,...,...,...,...
3466220,24852,3419531,2,1
3466221,24852,3419542,6,0
3466222,24852,3419629,5,1
3466223,24852,3420088,9,1


In [31]:
orders = raw_orders[raw_orders['order_id'].isin(order_products['order_id'].unique())]
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
1,2398795,1,prior,2,3,7,15.0
4,431534,1,prior,5,4,15,28.0
10,1187899,1,train,11,4,8,14.0
11,2168274,2,prior,1,2,11,NaN
12,1501582,2,prior,2,5,10,10.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


In [32]:
user_ids = orders["user_id"].unique()
product_ids = products["product_id"].unique()

dataset = Dataset()
dataset.fit(user_ids, product_ids)

user_mappings = dataset.mapping()[0]
item_mappings = dataset.mapping()[2]

len(user_mappings), len(item_mappings), list(user_mappings.items())[:5]

(173951, 20, [(1, 0), (2, 1), (3, 2), (5, 3), (6, 4)])

In [33]:
# Обратная матрица взаимодействий
inv_user_mappings = {v:k for k, v in user_mappings.items()}
inv_item_mappings = {v:k for k, v in item_mappings.items()}
list(inv_item_mappings.items())[:5]

[(0, 8277), (1, 40706), (2, 5876), (3, 28204), (4, 49683)]

In [34]:
# Создаем матрицу взаимодействий
data = orders.merge(order_products, how='inner', on='order_id').groupby(['user_id', 'product_id']).size()
data = data.to_frame().reset_index().rename(columns={0: 'weight'})
interactions, weights = dataset.build_interactions(data[['user_id', 'product_id', 'weight']].values)
interactions, weights

(<173951x20 sparse matrix of type '<class 'numpy.int32'>'
 	with 828811 stored elements in COOrdinate format>,
 <173951x20 sparse matrix of type '<class 'numpy.float32'>'
 	with 828811 stored elements in COOrdinate format>)

In [35]:
interactions.todense(), weights.todense()

(matrix([[0, 0, 0, ..., 0, 1, 0],
         [0, 0, 0, ..., 0, 1, 1],
         [0, 0, 0, ..., 1, 0, 0],
         ...,
         [0, 0, 0, ..., 1, 1, 0],
         [0, 1, 1, ..., 1, 1, 0],
         [0, 0, 0, ..., 0, 1, 1]], dtype=int32),
 matrix([[ 0.,  0.,  0., ...,  0.,  2.,  0.],
         [ 0.,  0.,  0., ...,  0.,  1.,  8.],
         [ 0.,  0.,  0., ...,  1.,  0.,  0.],
         ...,
         [ 0.,  0.,  0., ...,  1.,  8.,  0.],
         [ 0.,  5.,  4., ..., 15., 22.,  0.],
         [ 0.,  0.,  0., ...,  0.,  6.,  5.]], dtype=float32))

In [36]:
def evaluate_model(model):
    train, test = random_train_test_split(interactions, test_percentage=0.2, random_state=42)

    for metric in [precision_at_k, recall_at_k]:        
        print(f"{metric.__name__}:")
        print(f"\t- train: %.3f" % metric(model, train, k=5).mean())
        print(f"\t- test: %.3f" % metric(model, test, k=5).mean())
        print(f"\t- test new: %.3f" % metric(model, test, train, k=5).mean())

In [37]:
def objective(trial):

    train, test = random_train_test_split(interactions, test_percentage=0.2, random_state=42)

    param = {
        'no_components': trial.suggest_int("no_components", 5, 64),
        "learning_schedule": trial.suggest_categorical("learning_schedule", ["adagrad", "adadelta"]),
        "loss":  trial.suggest_categorical("loss", ["bpr", "warp", "warp-kos"]),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 1),
        "item_alpha": trial.suggest_float("item_alpha", 1e-10, 1e-06, log=True),
        "user_alpha": trial.suggest_float("user_alpha", 1e-10, 1e-06, log=True), 
        "max_sampled": trial.suggest_int("max_sampled", 5, 15),
    }
    epochs = trial.suggest_int("epochs", 20, 50)

    model = LightFM(**param, random_state=123)
    model.fit(train, epochs=epochs, verbose=True)

    val_precision = precision_at_k(model, test, k=5).mean()

    return val_precision

if True:
    best_params = {
        'no_components': 5,
        'learning_schedule': 'adagrad',
        'loss': 'warp',
        'learning_rate': 0.7510216830421954,
        'item_alpha': 6.201256477126263e-09,
        'user_alpha': 9.592173442318563e-09,
        'max_sampled': 12,
        'epochs': 46,
    }
    print("No learning")
    display(best_params)
else:
    study = optuna.create_study(direction="maximize")
    
    study.enqueue_trial(params={
        "no_components": 10, 
        "learning_schedule": 'adagrad', 
        "loss": 'warp',
        "learning_rate": 0.5,
        "item_alpha": 1e-9, 
        "user_alpha": 1e-8, 
        "max_sampled": 15,
        "epochs": 30,
    })
    
    study.optimize(objective, n_trials=50)
    
    best_params = study.best_params
    for k, v in best_params.items():
        print(k, ":", v)
    
    display(optuna.importance.get_param_importances(study))

No learning


{'no_components': 5,
 'learning_schedule': 'adagrad',
 'loss': 'warp',
 'learning_rate': 0.7510216830421954,
 'item_alpha': 6.201256477126263e-09,
 'user_alpha': 9.592173442318563e-09,
 'max_sampled': 12,
 'epochs': 46}

In [38]:
if 'epochs' in best_params:
    num_epochs = best_params['epochs']
    del best_params['epochs']

model = LightFM(**best_params, random_state=123)

model.fit(
    interactions,
    epochs=num_epochs,
    verbose=True
)

evaluate_model(model)

Epoch: 100%|████████████████████████████████████████████████████████████████████████████| 46/46 [00:20<00:00,  2.21it/s]


precision_at_k:
	- train: 0.503
	- test: 0.214
	- test new: 0.294
recall_at_k:
	- train: 0.761
	- test: 0.692
	- test new: 0.893


In [39]:
# Сравним с топ 5 рекомендацией
# todo сделать подсчет метрики для новых пользователей

k = 5
static_prediction = top_popular_product_counts[-k:]['product_id']
static_prediction_ind = list(map(lambda x: item_mappings[x], static_prediction))

def static_precision_at_k(interactions, suppress_interactions=None):
    interactions = interactions.copy()
    interactions.data = np.isin(interactions.col, static_prediction_ind)
    hit = np.squeeze(np.array(interactions.sum(axis=1)))
    return hit / k

def static_recall_at_k(interactions, suppress_interactions=None):
    retrieved = np.squeeze(interactions.getnnz(axis=1))

    interactions = interactions.copy()
    interactions.data = np.isin(interactions.col, static_prediction_ind)
    hit = np.squeeze(np.array(interactions.sum(axis=1)))

    hit = hit[retrieved > 0]
    retrieved = retrieved[retrieved > 0]

    return hit / retrieved


train, test = random_train_test_split(interactions, test_percentage=0.2, random_state=42)
for metric in [static_precision_at_k, static_recall_at_k]:        
    print(f"{metric.__name__}:")
    print(f"\t- train: %.3f" % metric(train).mean())
    print(f"\t- test: %.3f" % metric(test).mean())
    # print(f"\t- test new: %.3f" % metric(test, train).mean())

static_precision_at_k:
	- train: 0.280
	- test: 0.070
static_recall_at_k:
	- train: 0.413
	- test: 0.383


In [19]:
# Пример инференса
from inference.model import Model

recs_model = Model(model, dataset)
recs_product_ids = recs_model.get_recommendations()
recs_product_ids

[33731, 24852, 21903, 21137, 13176]

In [20]:
%%timeit
recs_model.get_recommendations()

4.7 ms ± 73.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
recs = products[products['product_id'].isin(recs_product_ids)]
recs

,product_id,count,product_name,aisle_id,department_id
33,33731,46932,Grated Parmesan,21,16
96,21903,251705,Organic Baby Spinach,123,4
97,21137,275577,Organic Strawberries,24,4
98,13176,394930,Bag of Organic Bananas,24,4
99,24852,491291,Banana,24,4


In [22]:
# Сохраним модель
with open('../artifacts/0-0-1.pickle', 'wb') as file:
    pickle.dump(recs_model, file, protocol=pickle.HIGHEST_PROTOCOL)